<a href="https://colab.research.google.com/github/joey0320/reversemethod/blob/master/REVERSE_METHOD_PIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install neptune-client

In [ ]:
import neptune

neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZjY2Y2M1MjktNzljYy00MTFjLWE1NTQtOTAxYTFhMmNjM2VhIn0=",
    project_qualified_name="joey0320/reverse-2"
)

Project(joey0320/reverse-2)

#Headers and Configs

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm
import time
import cv2


#use gpu    please somebody buy me a gpu .......
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

#Global variables
config = {
    'load_data':True,
    'load_test_data':True,

    'batch_size':64,
    'val_batch_size':64,
    'learning_rate':2e-4,
    'epochs':5,
    'patience':2,

    'img_size':64,
    'img_channels':4,

    'label_size':3,

    'output':3,

    'weight':torch.tensor([2.2, 2.2, 0.3]),
}
MODEL_NAME = f"model-{int(time.time())}"
neptune.create_experiment(name='rev1', params=config, upload_source_files=['libs/DataCaller.py',
                                                                           'libs/EarlyStopper.py',
                                                                           'libs/Net.py',
                                                                           'libs/train_test.py',
                                                                           'libs/utils.py'])


cuda
https://ui.neptune.ai/joey0320/reverse-2/e/REV1-143


Experiment(REV1-143)

# Data Reader
- read the data & check if loaded properly

In [ ]:
class DataCaller(Dataset):
    def __init__(self, data_path, input_size, label_size):
        
        self.data_path = data_path
        self.input_size = input_size
        self.label_size = label_size

        self.data = np.array(pd.read_csv(self.data_path, header=None), dtype=np.float)
        self.data = self.data[1:, 1:]
        self.X = self.data[:,:-self.label_size]
        self.Y = self.data[:,-self.label_size:]

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.X[idx,:]
        x = np.reshape(x, [4, self.input_size//4, self.input_size])
        x = torch.FloatTensor(x)
        y = torch.FloatTensor(self.Y[idx,:])
        return x, y

# DL Model

In [ ]:
class Net(nn.Module):
    def __init__(self, INPUT_CHANNELS, OUTPUT):
        super().__init__()

        self.INPUT_CHANNELS = INPUT_CHANNELS
        self.OUTPUT = OUTPUT

        self.cnn = nn.Sequential(
            nn.Conv2d(self.INPUT_CHANNELS, 6, kernel_size=3, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=(1, 2)),
            #3 8 16
            
            nn.Conv2d(6, 9, kernel_size=3, padding=1),
            nn.BatchNorm2d(9),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=(1, 2)),
            #6, 8, 8
            
            nn.Conv2d(9, 12, kernel_size=3, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            #9, 4, 4
            
            nn.Conv2d(12, 15, kernel_size=3, padding=1),
            nn.BatchNorm2d(15),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            #12, 2, 2

            nn.Conv2d(15, 18, kernel_size=3, padding=1),
            nn.BatchNorm2d(18),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
            #12, 2, 2
        )
        self.fcc = nn.Sequential(
            nn.Linear(36, self.OUTPUT)
            #nn.ReLU()
        )

    def forward(self, x):
        feature = self.cnn(x)
        #print(feature.size())
        feature = feature.view(feature.size(0), -1)
        output = self.fcc(feature)

        A = output[:,0]
        B = output[:,1]
        d = output[:,2]

        return A, B, d


#Early Stopper

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = - val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

#Utility functions

In [ ]:
def filter_input(x):
  x[x<0] = 0  


def weighted_mse(input, target, weight, device):
  input = input.to(device)
  target = target.to(device)
  weight = weight.to(device)
  return torch.sum(weight * (input - target)**2)

def WMSE4(a_, b_, d_, t_, a, b, d, t, ratio, weight, device):
  a, b, d, t = a.to(device), b.to(device), d.to(device), t.to(device)
  a_, b_, d_, t_ = a_.to(device), b_.to(device), d_.to(device), t_.to(device)
  weight = weight.to(device)

  a_, b_, d_, t_ = a_.squeeze(), b_.squeeze, d_.squeeze(), t_.squeeze()

  loss_fn = nn.MSELoss()
  loss = weight[0] * loss_fn(ratio * a, a_) + weight[1] * loss_fn(ratio * b, b_) + weight[2] * loss_fn(d, d_) + weight[3] * loss_fn(t, t_)
  return loss

def WMSE3(a_, b_, d_, a, b, d, weight, device):
  a, b, d = a.to(device), b.to(device), d.to(device)
  a_, b_, d_ = a_.to(device), b_.to(device), d_.to(device)
  weight = weight.to(device)
  a_, b_, d_ = a_.squeeze(), b_.squeeze(), d_.squeeze()

  loss_fn = nn.MSELoss()
  loss = weight[0] * loss_fn(a, a_) + weight[1] * loss_fn(b, b_) + weight[2] * loss_fn(d, d_)
  return loss

def WMSE2(a_, b_, d_, a, b, d, weight, device):
  a = a.to(device)
  b = b.to(device)
  d = d.to(device)
  a_ = a_.to(device)
  b_ = b_.to(device)
  d_ = d_.to(device)
  weight = weight.to(device)

  a = a.view(len(a))
  b = b.view(len(b))
  d = d.view(len(d))

  loss_fn = nn.MSELoss()

  loss = weight[0] * loss_fn(a, a_) + weight[1] * loss_fn(b, b_) + weight[2] * loss_fn(d, d_)
  return loss

def weighted_MSE(a, b, d, t, label_a, label_b, label_d, label_t, weight, device):
  a = a.to(device)
  b = b.to(device)
  d = d.to(device)
  t = t.to(device)

  label_a = label_a.to(device)
  label_b = label_b.to(device)
  label_d = label_d.to(device)
  label_t = label_t.to(device)

  weight = weight.to(device)
  
  a = a.view(len(a))
  b = b.view(len(b))
  d = d.view(len(d))
  t = t.view(len(t))

  crit1 = nn.MSELoss()
  crit2 = nn.MSELoss()
  crit3 = nn.MSELoss()
  crit4 = nn.MSELoss()

  return weight[0] * crit1(a, label_a) + weight[1] * crit2(b, label_b) + weight[2] * crit3(d, label_d) + weight[3] * crit4(t, label_t)

def save_plot(train_loss, valid_loss, limit):
	# visualize the loss as the network trained
	fig = plt.figure(figsize=(10,8))
	plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
	plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

	# find position of lowest validation loss
	minposs = valid_loss.index(min(valid_loss))+1 
	plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

	plt.xlabel('epochs')
	plt.ylabel('loss')
	plt.ylim(0, limit) # consistent scale
	plt.xlim(0, len(train_loss)+1) # consistent scale
	plt.grid(True)
	plt.legend()
	plt.tight_layout()
	plt.show()
	fig.savefig('loss_plot.png', bbox_inches='tight', dpi=300)

#train & test code

In [ ]:
import neptune
import torch
import torch.optim as optim
from tqdm import trange
import os
import pandas
from pandas import DataFrame
import time

#train the model
def train(net, trainloader, valloader, weight, EPOCHS, LEARNING_RATE, IMG_CHANNELS, IMG_SIZE, MODEL_NAME, patience, validate_every, device):
    
    lmd = lambda epoch:0.97

    optimizer = optim.Adam(net.parameters(), LEARNING_RATE, betas=(0.9, 0.999), eps=1e-09, weight_decay=0, amsgrad=False)
    scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lmd)

    train_losses = []
    val_losses = []
    avg_train_losses = []
    avg_val_losses = []

    early_stopping = EarlyStopping(patience=patience, verbose=True)

    with open("model.log", "a") as f:
        for epoch in trange(EPOCHS):

                net.train()
                for batch_idx, samples in enumerate(trainloader):
                    x, y = samples
                    x, y = x.to(device), y.to(device)

                    A, B, d = y[:, 0], y[:, 1], y[:, 2]    
                    A_, B_, d_ = net(x)

                    loss = WMSE2(A_, B_, d_, A, B, d, weight, device)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    train_losses.append(loss.item())

                    if batch_idx == 1:
                        print(A)
                        print(A_)
                        print(B)
                        print(B_)
                        print(d)
                        print(d_)

                net.eval()
                for batch_idx, samples in enumerate(valloader):
                    x, y = samples
                    x, y = x.to(device), y.to(device)

                    A, B, d = y[:, 0], y[:, 1], y[:, 2]
                    A_, B_, d_ = net(x)

                    loss = WMSE2(A_, B_, d_, A, B, d, weight, device)
                    val_losses.append(loss.item())

                train_loss = np.average(train_losses)
                valid_loss = np.average(val_losses)
                avg_train_losses.append(train_loss)
                avg_val_losses.append(valid_loss)
                train_losses = []
                val_losses = []

                neptune.log_metric('train loss', train_loss)
                neptune.log_metric('validation loss', valid_loss)

                f.write(f"{MODEL_NAME},{round(time.time(), 3)},  {round(float(train_loss), 4)},  {round(float(valid_loss),4)}\n")
                print("\nloss : ", train_loss, "val loss : ", valid_loss, "\n")
                
                early_stopping(valid_loss, net)
                
                if early_stopping.early_stop:
                    print("Early stopping")
                    break

                scheduler.step()

        net.load_state_dict(torch.load('checkpoint.pt'))
        neptune.log_artifact('checkpoint.pt')

    return avg_train_losses, avg_val_losses

def test(net, testloader, IMG_CHANNELS, IMG_SIZE, OUTPUT_LABEL_SIZE, device, pfx):
    predictions = []
    with torch.no_grad():
        for i, sample in enumerate(testloader):
            predict = []
            x ,y = sample            
            x, y = x.to(device), y.to(device)
            
            A_, B_, d_ = net(x)
            print(A_, B_, d_)
            print(y)

            final_result = torch.cat([A_, B_, d_])
            final_result = final_result.to("cpu")
          
            predictions.append(final_result.numpy())

    print(predictions)
    
    predictions = np.array(predictions)
    predictions = predictions.reshape(-1, OUTPUT_LABEL_SIZE)
    df = DataFrame(predictions)
    file_name = pfx + 'predictions.xlsx'
    df.to_excel(file_name, header=None, index=None)
    neptune.log_artifact(file_name)
    return predictions

#Beam Formation Code

In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits import mplot3d

In [ ]:

PI = 3.14159265358979
wave_length = 1.0
k = 2*PI / wave_length
n_ = 120*PI
num = 64
r = 10 * wave_length
EPS = 1e-3
dx = wave_length/10


class Surface():
    def __init__(self, nx, ny):
        self.nx = nx
        self.ny = ny
        self.dir = np.zeros((num//4, num))
    
    """------------------  from here call after calling get_dir ------------------------"""
    
    def get_maxdir(self):
        return np.max(self.dir)
    
    def get_hpbw(self):
        dir_ = self.rotate2origin()
        i, _ = np.unravel_index(np.argmax(dir_, axis=None), dir_.shape)
        hp = self.get_maxdir() - 3
        dir_[dir_<hp] = 0
        t_cnt = np.count_nonzero(dir_[:,0])
        return t_cnt*2*PI/num

    def get_space_angle(self):
        sa = 0
        for i in range(num//4):
            for j in range(num):
                if self.dir[i, j] >= self.get_maxdir() - 3:
                    sa += np.sin(2*PI/num*i) * (4*PI**2)/(num**2)
        return sa

    def get_info(self):
        return self.get_maxdir(), self.get_hpbw(), self.get_space_angle()

    def rotate2origin(self):
        i, j = np.unravel_index(np.argmax(self.dir, axis=None), self.dir.shape)
        dir_ = np.copy(self.dir)
        dir_ = np.roll(dir_, -j, axis=1)
        return dir_
    
    def plot_dir(self):
        d = self.dir
        theta = np.linspace(0, PI/2, num//4)
        phi = np.linspace(0, PI*2, num)
        phi_, theta_ = np.meshgrid(phi, theta)
        x = d * np.vstack(np.sin(theta_)) * np.cos(phi_)
        y = d * np.vstack(np.sin(theta_)) * np.sin(phi_)
        z = d * np.vstack(np.cos(theta_))
        
        dmax = np.max(d)
        i, j = np.unravel_index(np.argmax(d, axis=None), d.shape)

        ax = plt.axes(projection='3d')
        ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
        ax.set_title('surface');
        ax.set_xlabel('$X$')
        ax.set_ylabel('$Y$')
        ax.set_xlim([-35, 35])
        ax.set_ylim([-35, 35])
        ax.set_zlim(bottom = 0.0)
    
    def plot_plane_dir(self):
        i, j = np.unravel_index(np.argmax(self.dir, axis=None), self.dir.shape)
        theta = np.linspace(0, PI/2, num//4)
        phi = np.linspace(0, 2*PI, num)
        
        fig, axes = plt.subplots(nrows=1, ncols=2)

        plt.subplot(121)
        plt.plot(theta, self.dir[:,j])
        plt.xlabel('theta (rad)')
        plt.ylabel('Directivity Theta Plane (dB)')
        
        plt.subplot(122)
        plt.plot(phi, self.dir[i,:])
        plt.xlabel('phi (rad)')
        plt.ylabel('Directivity Phi Plane (dB)')

        fig.tight_layout() # Or equivalently,  "plt.tight_layout()"
        #plt.show()
    def plot_xz_plane(self):
        i, j = np.unravel_index(np.argmax(self.dir, axis=None), self.dir.shape)
        theta = np.linspace(0, PI/2, num//4)
        phi = np.linspace(0, 2*PI, num)
        plt.plot(theta, self.dir[:,j])
        plt.xlabel('theta (rad)')
        plt.ylabel('Directivity Theta Plane (dB)')

    def plot_yz_plane(self):
        i, j = np.unravel_index(np.argmax(self.dir, axis=None), self.dir.shape)
        theta = np.linspace(0, PI/2, num//4)
        phi = np.linspace(0, 2*PI, num)
        plt.plot(phi, self.dir[i,:])
        plt.xlabel('phi (rad)')
        plt.ylabel('Directivity Phi Plane (dB)')

"""
metasurface with continuous E field distribution

nx, ny : number of pixels in x, y direction
dx : size of pixel
"""

class MetaSurfaceAperture(Surface):
    def __init__(self, ax, ay):
        nx = int(ax//dx)
        ny = int(ay//dx)
        
        super(MetaSurfaceAperture, self).__init__(nx, ny)
        self.dx = dx
        self.dy = dx
        self.ax = ax
        self.ay = ay
    
    def set_zero_field(self):
        self.ex = np.zeros((self.nx, self.ny), dtype=np.complex128)
        self.ey = np.zeros((self.nx, self.ny), dtype=np.complex128)
        self.hx = np.zeros((self.nx, self.ny), dtype=np.complex128)
        self.hy = np.zeros((self.nx, self.ny), dtype=np.complex128)

    def set_uniform_e_field(self):
        self.set_zero_field()
        self.ex = np.ones((self.nx, self.ny))
    
    def set_horn_e_field(self, d):
        a = 1.0
        b = 1.0
        self.set_zero_field()
        for i in range(self.nx):
            for j in range(self.ny):
                x = self.ax*i/self.nx - self.ax/2
                y = self.ay*j/self.ny - self.ay/2
                r = np.sqrt(x**2 + y**2)
                R = np.sqrt(r**2 + d**2)

                cos_theta = d / R
                sin_theta = r / R

                if r == 0:
                    cos_phi = 1
                    sin_phi = 1
                else:
                    cos_phi = x/r
                    sin_phi = y/r

                coeff = 1j*a*b*k*np.exp(-1j*k*R)/(2*PI*R)
                X = k*a/2*sin_theta*cos_phi
                Y = k*b/2*sin_theta*sin_phi
                if abs(X) < EPS:
                    X = 1
                else:
                    X = np.sin(X)/X
                if abs(Y) < EPS:
                    Y = 1
                else:
                    Y = np.sin(Y)/Y
                F = X*Y
                et = 0.5*coeff*sin_phi*(1+cos_theta)*F
                ep = 0.5*coeff*cos_phi*(1+cos_theta)*F
                ht = -ep/n_
                hp = et/n_
                self.ex[i, j] = et*cos_theta*cos_phi - ep*sin_phi
                self.ey[i, j] = et*cos_theta*sin_phi + ep*cos_phi
                self.hx[i, j] = ht*cos_theta*cos_phi - hp*sin_phi
                self.hy[i, j] = ht*cos_theta*sin_phi + hp*cos_phi

    def set_dipole_e_field(self, d):
        self.set_zero_field()
        coeff = 1j*60
        print(d)
        for i in range(self.nx):
            for j in range(self.ny):
                x = self.ax * i / self.nx - self.ax/2
                y = self.ay * j / self.ny - self.ay/2
                r = np.sqrt(d**2 + y**2)
                R = np.sqrt(r**2 + x**2)
                cos_theta = x / R
                sin_theta = r / R
                cos_phi = y / r
                sin_phi = d / r
                F = np.cos(PI / 2 * cos_theta) / sin_theta;
                AF = (1 + 2 * np.cos(PI * sin_theta * cos_phi)) * (1 + 2 * np.cos(PI * sin_theta * sin_phi));
       
                F = F * AF;
                H_phi = 1/n_ * np.exp(- 1j * k * R) / R  * F;
                E_theta = np.exp(- 1j * k * R) / R * F;
            
                self.hx[i, j] = 0;
                self.hy[i, j] = - coeff * H_phi * sin_phi;
                self.ex[i, j] = - coeff * E_theta * sin_theta;
                self.ey[i, j] = coeff * E_theta * cos_theta * cos_phi;
    
    def set_phase(self, theta0, phi0):
        px = np.sin(theta0)*np.cos(phi0)
        py = np.sin(theta0)*np.sin(phi0)
        xid = np.arange(0, self.nx, 1)
        yid = np.arange(0, self.ny, 1)
        x_expon = np.exp(-1j*k*self.dx*px*xid)
        y_expon = np.exp(-1j*k*self.dy*py*yid)
        self.ex = self.ex*(np.vstack(x_expon)*y_expon)
        self.ey = self.ey*(np.vstack(x_expon)*y_expon)
        self.hx = self.hx*(np.vstack(x_expon)*y_expon)
        self.hy = self.hy*(np.vstack(x_expon)*y_expon)

    def equivalence_theorem(self):
        mx = self.ey
        my = -self.ex
        jx = -self.hy
        jy = self.hx
        return (mx, my, jx, jy)

    def get_far_field(self, theta0, phi0):
        self.set_phase(theta0, phi0)
        mx, my, jx, jy = self.equivalence_theorem()
        n_phi   = np.zeros((num//4, num), dtype=np.complex128)
        n_theta = np.zeros((num//4, num), dtype=np.complex128)
        l_phi   = np.zeros((num//4, num), dtype=np.complex128)
        l_theta = np.zeros((num//4, num), dtype=np.complex128)

        theta = np.linspace(0, PI/2, num//4)
        phi = np.linspace(0, 2*PI, num)
        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)
        sin_phi = np.sin(phi)
        cos_phi = np.cos(phi)

        for i in range(self.nx):
            for j in range(self.ny):
                exponent = 1j * k * ( (i - num/2)* self.dx * np.vstack(sin_theta) * cos_phi  \
                                    + (j - num/2)* self.dy * np.vstack(sin_theta) * sin_phi)
                exponent = np.exp(exponent) * self.dx * self.dy
                n_theta += (jx[i, j] * np.vstack(cos_theta) * cos_phi + jy[i, j] * np.vstack(cos_theta) * sin_phi) * exponent
                n_phi += (- jx[i, j] * sin_phi + jy[i, j] * cos_phi) * exponent
                l_theta += (mx[i, j] * np.vstack(cos_theta) * cos_phi + my[i, j] * np.vstack(cos_theta) * sin_phi) * exponent
                l_phi += (- mx[i, j] * sin_phi + my[i, j] * cos_phi) * exponent
        return (l_phi + n_theta*n_, -l_theta + n_phi*n_)


    def get_dir(self, theta0, phi0):
        theta = np.linspace(0, PI/2, num//4)
        sin_theta = np.vstack(np.sin(theta))
        
        e_theta, e_phi = self.get_far_field(theta0, phi0)
        u = np.real(e_theta*np.conj(e_theta) + e_phi*np.conj(e_phi))
        pr = 1/(4*PI)*np.sum(u*sin_theta*(4*PI*PI/(num**2))) 
        self.dir = 10*np.log10(u/pr)
        self.dir[self.dir<0] = 0

#Make predict beams

In [ ]:
def make_predict_beams(predictions, testloader, name_prefix):
    cnt = np.shape(predictions)[0]

    results = np.zeros((cnt, 24))

    for i, sample in enumerate(testloader):
        print(f'{i} th test data')
        x, y = sample
        A = y[:,0]
        B = y[:,1]
        d = y[:,2]
        A_ = predictions[i,0]
        B_ = predictions[i,1]
        d_ = predictions[i,2]

        ms = MetaSurfaceAperture(A_.item(), B_.item())
        ms.set_horn_e_field(d_.item())

        ms_test = MetaSurfaceAperture(A.item(), B.item())
        if name_prefix == 'uniform':
            ms_test.set_uniform_e_field()
        else:
            ms_test.set_dipole_e_field(d.item())


        dir_maxes = []
        hpbw_ts = []
        sas = []

        test_dir_maxes = []
        test_hpbw_ts = []
        test_sas = []

        for j in range(4):
            theta = theta_max*j/3
            ms.get_dir(theta, 0)
            dir_max, hpbw_t, sa = ms.get_info()
            dir_maxes.append(dir_max)
            hpbw_ts.append(hpbw_t)
            sas.append(sa)

            ms_test.get_dir(theta, 0)
            test_dir_max, test_hpbw_t, test_sa = ms_test.get_info()
            test_dir_maxes.append(test_dir_max)
            test_hpbw_ts.append(test_hpbw_t)
            test_sas.append(test_sa)
        result = np.array([test_dir_maxes, test_hpbw_ts, test_sas, dir_maxes, hpbw_ts, sas])
        print(result)
        result = np.reshape(result, [1, -1])
        results[i,:] = result
    
    df = pd.DataFrame(results)
    result_name = name_prefix + '_test_results.csv'
    df.to_csv(result_name)
    neptune.log_artifact(result_name)
    return results

In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits import mplot3d


#Result Analysis

In [ ]:
def result_analysis(results, pfx):
    data_points = np.shape(results)[0]

    # parse results
    dmax = results[:,0:4]
    dmax_ = results[:,12:16]
    hpbw = results[:,4:8]
    hpbw_ = results[:,16:20]
    sa = results[:,8:12]
    sa_ = results[:,20:]
    hpbw_p = sa / hpbw
    hpbw_p_ = sa_ / hpbw_

    # calculate error
    d_error = np.abs(np.mean((dmax - dmax_), axis=1))
    h_error = np.abs(np.mean((hpbw - hpbw_)/(hpbw+1e-6)*100, axis=1))
    s_error = np.abs(np.mean((sa - sa_)/(sa+1e-6)*100, axis=1))
    hp_error = np.abs(np.mean((hpbw_p - hpbw_p_)/(hpbw_p+1e-6)*100, axis=1))

    # set cdf intervals
    d_interval = 0.5
    d_error_lim = 5.0
    d_error_min = 0.0

    h_interval = 5.0
    h_error_lim = 60.0
    h_error_min = 0.0

    s_interval = 5.0
    s_error_lim = 60.0
    s_error_min = 0.0

    hp_interval = 5.0
    hp_error_lim = 60.0
    hp_error_min = 0.0

    # get cdf error ranges
    d_error_max = np.arange(d_error_min, d_error_lim, d_interval)
    h_error_max = np.arange(h_error_min, h_error_lim, h_interval)
    s_error_max = np.arange(s_error_min, s_error_lim, s_interval)
    hp_error_max = np.arange(hp_error_min, hp_error_lim, hp_interval)

    # set cdf to one
    d_len = np.shape(d_error_max)[0]
    h_len = np.shape(h_error_max)[0]
    s_len = np.shape(s_error_max)[0]
    hp_len = np.shape(hp_error_max)[0]

    cdf = np.zeros((d_len, h_len, hp_len), dtype=np.float32)

    # calculate cdf
    for i in range(d_len):
        for j in range(h_len):
            for k in range(hp_len):
                idx = np.ones(data_points)
                idx[d_error > d_error_max[i]] = 0
                idx[h_error > h_error_max[j]] = 0
                idx[hp_error > hp_error_max[k]] = 0
                cdf[i, j, k] = np.count_nonzero(idx)/data_points*100

    d_lim = 2
    di = np.where(d_error_max == d_lim)
    cur_cdf = cdf[di[0][0],:,:]

    # export to excel
    error_data = pd.DataFrame(cur_cdf, columns=list(h_error_max), index=list(h_error_max))
    xlsx_dir = pfx + str(d_lim) + '.xlsx'
    error_data.to_excel(xlsx_dir, sheet_name='1')
    neptune.log_artifact(xlsx_dir)

    def plot_cdf(cur_cdf):
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        
        X = np.copy(s_error_max)
        Y = np.copy(h_error_max)
        Y, X = np.meshgrid(Y, X)
        
        ax.plot_surface(X, Y, cur_cdf, cmap=cm.coolwarm)
        ax.view_init(25, 225)
        ax.set_zlabel('Cumulative Distribution (%)')
        plt.xlabel('Relative HPBW (phi) Error (%)')
        plt.ylabel('Relative HPBW (theta) Error (%)')
        plt.tick_params(axis='both', which='minor', labelsize=6)
        plt.rc('xtick',labelsize=6)
        plt.rc('ytick',labelsize=6)
        plt.xticks(np.arange(5, h_error_lim, 5.0))
        plt.yticks(np.arange(5, h_error_lim, 5.0))

        plt.show()

    #plot & save figure of cdf
    plot_cdf(cur_cdf)
    plot_name = pfx + str(d_lim)
    plt.savefig(plot_name, dpi=300)
    neptune.log_artifact(plot_name+'.png')
    print(np.max(cur_cdf))
    
    d_error_list = list(d_error)
    medIdx = d_error_list.index(np.percentile(d_error_list, 50, interpolation='nearest'))

    return medIdx

#save 2d plots for beams with median Dmax error
def save_2d_plots(predictions, testloader, median_idx, pfx, phase_pfx):
    for i, sample in enumerate(testloader):
        if i != median_idx:
            continue
        print(i)
        x, y = sample
        A = y[:,0]
        B = y[:,1]
        d = y[:,2]
        A_ = predictions[i,0]
        B_ = predictions[i,1]
        d_ = predictions[i,2]

        ms = MetaSurfaceAperture(A_.item(), B_.item())
        ms.set_horn_e_field(d_.item())

        ms_test = MetaSurfaceAperture(A.item(), B.item())
        if pfx == 'uniform':
            ms_test.set_uniform_e_field()
        else:
            ms_test.set_dipole_e_field(d.item())

        for j in range(4):
            theta = theta_max*j/3
            ms.get_dir(theta, 0)
            ms_test.get_dir(theta, 0)

            plt.close('all')
            plot_name = 'xz_' + pfx + phase_pfx + '_recover_and_test_beam-angle_' + str(j)
            ms.plot_xz_plane()
            ms_test.plot_xz_plane()
            plt.legend(['Recover Beam', 'Target Beam'], loc='upper right')
            plt.show()
            plt.savefig(plot_name, dpi=300)
            neptune.log_artifact(plot_name+'.png')

            plt.close('all')
            plot_name = 'yz_' + pfx + phase_pfx + '_recover_and_test_beam-angle_' + str(j)
            ms.plot_yz_plane()
            ms_test.plot_yz_plane()
            plt.legend(['Recover Beam', 'Target Beam'], loc='upper right')
            plt.show()
            plt.savefig(plot_name, dpi=300)
            neptune.log_artifact(plot_name+'.png')

#Grid Search Code

In [ ]:
def grid_search(x, hornloader):
    loss_fn = nn.MSELoss()
    loss_min = 1e12
    y_opt = torch.zeros(3)
    for i, sample in enumerate(hornloader):
        x_, y_ = sample
        loss = loss_fn(x, x_)
        if loss.item() < loss_min:
            loss_min = loss.item()
            y_opt = y_

    A_, B_, d_ = y_opt[:,0], y_opt[:,1], y_opt[:,2]
    return A_, B_, d_

def get_grid_search_results(testloader, hornloader, name_prefix):
    grid_predictions = []
    cnt = len(testloader)
    results = np.zeros((cnt, 24))
    for i, sample in enumerate(testloader):
        x, y = sample
        A = y[:,0]
        B = y[:,1]
        d = y[:,2]
        A_, B_, d_ = grid_search(x, hornloader)
        grid_predictions.append([A_, B_, d_])

        print(A_)
        print(B_)
        print(d_)
        ms = MetaSurfaceAperture(A_.item(), B_.item())
        ms.set_horn_e_field(d_.item())

        ms_test = MetaSurfaceAperture(A.item(), B.item())
        if name_prefix == 'uniform':
            ms_test.set_uniform_e_field()
        else:
            ms_test.set_dipole_e_field(d.item())


        dir_maxes = []
        hpbw_ts = []
        sas = []

        test_dir_maxes = []
        test_hpbw_ts = []
        test_sas = []

        for j in range(4):
            theta = theta_max*j/3
            ms.get_dir(theta, 0)
            dir_max, hpbw_t, sa = ms.get_info()
            dir_maxes.append(dir_max)
            hpbw_ts.append(hpbw_t)
            sas.append(sa)

            ms_test.get_dir(theta, 0)
            test_dir_max, test_hpbw_t, test_sa = ms_test.get_info()
            test_dir_maxes.append(test_dir_max)
            test_hpbw_ts.append(test_hpbw_t)
            test_sas.append(test_sa)
        result = np.array([test_dir_maxes, test_hpbw_ts, test_sas, dir_maxes, hpbw_ts, sas])
        print(result)
        result = np.reshape(result, [1, -1])
        results[i,:] = result
    
    df = pd.DataFrame(results)
    result_name = name_prefix + '_grid_search_results.csv'
    df.to_csv(result_name)
    neptune.log_artifact(result_name)

    grid_predictions = np.reshape(np.array(grid_predictions), [-1, 3])

    return results, grid_predictions


#Code Running Section
> result_analysis : exports cdf with MDE of less than 2dB error & returns median index sorted by MDE
> save_2d_plots : save xz, yz plane beam plots of median

#Data Loading Section

In [ ]:
!ls
!rm *.png *.xlsx *.csv *.pt

drive  sample_data
rm: cannot remove '*.png': No such file or directory
rm: cannot remove '*.xlsx': No such file or directory
rm: cannot remove '*.csv': No such file or directory
rm: cannot remove '*.pt': No such file or directory


In [ ]:
BASE_PATH = '/content/drive/My Drive/RM2/data1'
TRAIN_DATA_PATH = os.path.join(BASE_PATH, 'train')
VAL_DATA_PATH = os.path.join(BASE_PATH, 'val')
TEST_DATA_PATH = os.path.join(BASE_PATH, 'test_uniform')

In [ ]:
if config['load_data']:
    trainsets = []
    valsets = []
    hornsets = []
    for f in tqdm(os.listdir(TRAIN_DATA_PATH)):
      path = os.path.join(TRAIN_DATA_PATH, f)
      trainset = DataCaller(path, config['img_size'], config['label_size'])
      print(trainset.__len__())
      trainsets.append(trainset)
      hornsets.append(trainset)

    for f in tqdm(os.listdir(VAL_DATA_PATH)):
      path  = os.path.join(VAL_DATA_PATH, f)
      valset = DataCaller(path, config['img_size'], config['label_size'])
      print(valset.__len__())
      valsets.append(valset)
      hornsets.append(valset)

if config['load_test_data']:
    testsets = []
    for f in tqdm(os.listdir(TEST_DATA_PATH)):
      path  = os.path.join(TEST_DATA_PATH, f)
      testset = DataCaller(path, config['img_size'], config['label_size'])
      print(testset.__len__())
      testsets.append(testset)
    testset = ConcatDataset(testsets)
    print(testset.__len__())  
    
trainset = ConcatDataset(trainsets)
print(trainset.__len__())
valset = ConcatDataset(valsets)
print(valset.__len__())
hornset = ConcatDataset(hornsets)
print(hornset.__len__())


In [ ]:
#define dataloaders
trainloader = DataLoader(trainset, batch_size = config['batch_size'], shuffle=True)
valloader = DataLoader(valset, batch_size=config['batch_size'], shuffle=True)

testloader = DataLoader(testset, batch_size=1, shuffle=False)
hornloader = DataLoader(hornset, batch_size=1, shuffle=False)

In [ ]:
cnt = 0
for i, data in enumerate(hornloader):
    x, y = data
    x = np.array(x)
    x = x.reshape(-1, config['img_channels'], config['img_size']//4, config['img_size'])

    #print(x[0])
    print(x[0].shape)

    print(y)
    cnt += 1
    if cnt==1:
      break

(4, 16, 64)
tensor([[1., 1., 1.]])


#Train the model
- fit to horn antenna source

In [ ]:
net = Net(config['img_channels'], config['label_size'])
net.to(device)
print(net)


train_losses, valid_losses = train(net, trainloader, valloader, config['weight'], config['epochs'], config['learning_rate'], 
                                   config['img_channels'], config['img_size'], MODEL_NAME, 
                                   config['patience'], 1, device)


graph_limit = 5
save_plot(train_losses, valid_losses, graph_limit)
neptune.log_artifact('loss_plot.png')

#Test model on uniform test data

In [ ]:
net = Net(config['img_channels'], config['label_size'])
net.to(device)
print(net)

net.load_state_dict(torch.load('checkpoint.pt'))
net.eval()
uniform_predictions = test(net, testloader, config['img_channels'], config['img_size'], config['label_size'], device, 'uniform')

In [ ]:
theta_max = PI/4
pfx = 'uniform'
uniform_results = make_predict_beams(uniform_predictions, testloader, pfx)

In [ ]:
median_idx = result_analysis(uniform_results, 'uniform_DL')
save_2d_plots(uniform_predictions, testloader, median_idx, 'uniform', 'DL')

<IPython.core.display.Javascript object>

89.0
77


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Grid Search on uniform test data


In [ ]:
uniform_grid_search_results, uniform_grid_search_predictions = get_grid_search_results(testloader, hornloader, 'uniform')

In [ ]:
median_idx = result_analysis(uniform_grid_search_results, 'uniform_grid_search')
save_2d_plots(uniform_grid_search_predictions, testloader, median_idx, 'uniform', 'grid')

<IPython.core.display.Javascript object>

99.0
9


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Load Dipole Test data

In [ ]:
TEST_DATA_PATH2 = os.path.join(BASE_PATH, 'test_dipole')
if config['load_test_data']:
    testsets2 = []
    for f in tqdm(os.listdir(TEST_DATA_PATH2)):
      path  = os.path.join(TEST_DATA_PATH2, f)
      testset2 = DataCaller(path, config['img_size'], config['label_size'])
      print(testset2.__len__())
      testsets2.append(testset2)
    testset2 = ConcatDataset(testsets2)
    print(testset2.__len__())  

 20%|██        | 1/5 [00:01<00:06,  1.54s/it]

100


 40%|████      | 2/5 [00:03<00:04,  1.52s/it]

100


 60%|██████    | 3/5 [00:04<00:03,  1.52s/it]

100


 80%|████████  | 4/5 [00:06<00:01,  1.52s/it]

100


100%|██████████| 5/5 [00:07<00:00,  1.48s/it]

100
500


In [ ]:
testloader2 = DataLoader(testset2, batch_size=1, shuffle=False)

#Test Model on Dipole test dataset

In [ ]:
net = Net(config['img_channels'], config['label_size'])
net.load_state_dict(torch.load('checkpoint.pt'))
net.to(device)
net.eval()
print(net)


predictions_dipole = test(net, testloader2, config['img_channels'], config['img_size'], config['label_size'], device, 'dipole')

In [ ]:
theta_max = PI/4

pfx = 'dipole'
predictions_dipole = np.array(predictions_dipole)
results_dipole = make_predict_beams(predictions_dipole, testloader2, pfx)

In [ ]:
median_idx = result_analysis(results_dipole, 'dipole_DL')
save_2d_plots(predictions_dipole, testloader2, median_idx, 'dipole', 'DL')

#Grid Search on Dipole test dataset

In [ ]:
dipole_grid_search_results, dipole_grid_search_predictions = get_grid_search_results(testloader2, hornloader, 'dipole')

In [ ]:
median_idx = result_analysis(dipole_grid_search_results, 'dipole_grid_search')
save_2d_plots(dipole_grid_search_predictions, testloader2, median_idx, 'dipole', 'grid')

#Load data for 2 angles
> do separately : from above
> or else it will overwrite checkpoints, etc

In [ ]:
TRAIN_DATA_PATH2 = os.path.join(BASE_PATH, 'train_2angle')
VAL_DATA_PATH2 = os.path.join(BASE_PATH, 'val_2angle')
TEST_DATA_PATH3 = os.path.join(BASE_PATH, 'test_uniform_2angle')

if config['load_data']:
    trainsets2 = []
    for f in tqdm(os.listdir(TRAIN_DATA_PATH2)):
      path  = os.path.join(TRAIN_DATA_PATH2, f)
      trainset2 = DataCaller(path, config['img_size'], config['label_size'])
      print(trainset2.__len__())
      trainsets2.append(trainset2)
    trainset2 = ConcatDataset(trainsets2)
    print(trainset2.__len__())  

    valsets2 = []
    for f in tqdm(os.listdir(VAL_DATA_PATH2)):
        path = os.path.join(VAL_DATA_PATH2, f)
        valset2 = DataCaller(path, config['img_size'], config['label_size'])
        print(len(valset2))
        valsets2.append(valset2)
    valset2 = ConcatDataset(valsets2)
    print(len(valset2))

if config['load_test_data']:
    testsets3 = []
    for f in tqdm(os.listdir(TEST_DATA_PATH3)):
        path = os.path.join(TEST_DATA_PATH3, f)
        testset3 = DataCaller(path, config['img_size'], config['label_size'])
        print(len(testset3))
        testsets3.append(testset3)
    testset3 = ConcatDataset(testsets3)
    print(len(testset3))

In [ ]:
trainloader2 = DataLoader(trainset2, batch_size=config['batch_size'], shuffle=True)
valloader2 = DataLoader(valset2, batch_size=config['batch_size'], shuffle=True)
testloader3 = DataLoader(testset3, batch_size=1, shuffle=False)

#Train model on with data on 2 angles

In [ ]:
MODEL_NAME2 = f"model-{int(time.time())}"
net2 = Net(config['img_channels'], config['label_size'])
net2.to(device)
print(net2)
train_losses2, valid_losses2 = train(net2, trainloader2, valloader2, config['weight'], config['epochs'], config['learning_rate'], 
                                   config['img_channels'], config['img_size'], MODEL_NAME2, 
                                   config['patience'], 1, device)

graph_limit = 5
save_plot(train_losses2, valid_losses2, graph_limit)
#neptune.log_artifact('loss_plot.png')

In [ ]:
net2.load_state_dict(torch.load('checkpoint.pt'))
net2 = net2.to(device)
net2.eval()
predictions_uniform = test(net2, testloader3, config['img_channels'], config['img_size'], config['label_size'], device, 'dipole')

In [ ]:
theta_max = PI/4
pfx = 'uniform'
uniform_results = make_predict_beams(predictions_uniform, testloader3, pfx)

In [ ]:
median_idx = result_analysis(uniform_results, 'uniform_DL_2angle')
save_2d_plots(predictions_uniform, testloader, median_idx, 'uniform', 'DL_2angle')

<IPython.core.display.Javascript object>

78.0
25


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Compare time

In [ ]:
import time

net = net.to('cpu')
dl_times = []
grid_times = []
for i, sample in enumerate(testloader):
    x, y = sample
    start_time = time.time()
    net(x)
    end_time = time.time()
    dl_time = end_time - start_time
    dl_times.append(dl_time)

    start_time = time.time()
    grid_search(x, hornloader)
    end_time = time.time()
    grid_time = end_time - start_time
    grid_times.append(grid_time)

    if i == 100:
        break

times = np.reshape(np.array([dl_times, grid_times]), [2, -1])
time_df = pd.DataFrame(times)
time_file = 'computation_time.csv'
time_df.to_csv(time_file)
neptune.log_artifact(time_file)

print("DL average time %s seconds" % (sum(dl_times)/len(dl_times)))
print("Grid average time %s seconds" % (sum(grid_times)/len(grid_times)))

#Get CPU Info

In [ ]:
!cat /proc/cpuinfo > cpu_info.txt

In [ ]:
neptune.log_artifact('cpu_info.txt')